## Learning Objectives

* Showing Different Fractals with DPC++

* Understand the __Data Parallel C++ (DPC++)__ language and programming model



## Fractals and Mandelbrot
It is one of the most amazing discoveries in the realm of mathematics that not only does the simple equation Zn+1 = Zn2 + C create the infinitely complex Mandelbrot Set, but we can also find the same iconic shape in the patterns created by many other equations. In fact, the phenomenon of Mandelbrot Universality means that anytime we iterate a function that in some portion, at some scale, resembles the parabolic function Z2, then we will find small copies of the Mandelbrot Set in the map of that function.

### oneAPI Distribution
Intel&reg; oneAPI toolkits are available via multiple distribution channels:
* Local product installation: install the oneAPI toolkits from the __Intel® Developer Zone__.
* Install from containers or repositories: install the oneAPI toolkits from one of several supported
containers or repositories.
* Pre-installed in the __Intel® DevCloud__: a free development sandbox for access to the latest Intel® SVMS hardware and select oneAPI toolkits. 

## Tricorn(The Conjugate of mandelbrot)
Conjugate of mandelbrot
the tricorn, sometimes called the Mandelbar set, is a fractal defined in a similar way to the Mandelbrot set, but using the mapping 

Conj(Z^2 + C)

Where Z is the complex number


<img src="Assets/mandelbrot1.png">


## Introducing oneAPI
__oneAPI__ is a solution to deliver unified programming model to __simplify development__ across diverse architectures. It includes a unified and simplified language and libraries for expressing __parallelism__ and delivers uncompromised native high-level language performance across a range of hardware including __CPUs, GPUs, FPGAs__. oneAPI initiative is based on __industry standards and open specifications__ and is interoperable with existing HPC programming models.

<img src="Assets/oneapi2.png">


***


##  Editing the simple.cpp code
The Jupyter cell below with the gray background can be edited in-place and saved.

The first line of the cell contains the command **%%writefile 'simple.cpp'** This tells the input cell to save the contents of the cell into a file named 'simple.cpp' in your current directory (usually your home directory). As you edit the cell and run it in the Jupyter notebook, it will save your changes into that file.

The code below is some simple DPC++ code to get you started in the DevCloud environment. Simply inspect the code - there are no modifications necessary. Run the first cell to create the file, then run the cell below it to compile and execute the code.
1. Inspect the code cell below, then click run ▶ to save the code to a file
2. Run ▶ the cell in the __Build and Run__ section below the code snippet to compile and execute the code in the saved file

In [3]:
%%writefile lab/simple.cpp
//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

#include <chrono>
#include <iomanip>
#include <iostream>

// dpc_common.hpp can be found in the dev-utilities include folder.
// e.g., $ONEAPI_ROOT/dev-utilities/<version>/include/dpc_common.hpp
#include "dpc_common.hpp"
#include "mandel.hpp"

using namespace std;
using namespace sycl;

void ShowDevice(queue &q) {
  // Output platform and device information.
  auto device = q.get_device();
  auto p_name = device.get_platform().get_info<info::platform::name>();
  cout << std::setw(20) << "Platform Name: " << p_name << "\n";
  auto p_version = device.get_platform().get_info<info::platform::version>();
  cout << std::setw(20) << "Platform Version: " << p_version << "\n";
  auto d_name = device.get_info<info::device::name>();
  cout << std::setw(20) << "Device Name: " << d_name << "\n";
  auto max_work_group = device.get_info<info::device::max_work_group_size>();
  cout << std::setw(20) << "Max Work Group: " << max_work_group << "\n";
  auto max_compute_units = device.get_info<info::device::max_compute_units>();
  cout << std::setw(20) << "Max Compute Units: " << max_compute_units << "\n\n";
}

void Execute(queue &q) {
  // Demonstrate the Mandelbrot calculation serial and parallel.
#ifdef MANDELBROT_USM
  cout << "Parallel Mandelbrot set using USM.\n";
  MandelParallelUsm m_par(row_size, col_size, max_iterations, &q);
#else
  cout << "Parallel Mandelbrot set using buffers.\n";
  MandelParallel m_par(row_size, col_size, max_iterations);
#endif

  MandelSerial m_ser(row_size, col_size, max_iterations);

  // Run the code once to trigger JIT.
  m_par.Evaluate(q);

  // Run the parallel version and time it.
  dpc_common::TimeInterval t_par;
  for (int i = 0; i < repetitions; ++i) m_par.Evaluate(q);
  double parallel_time = t_par.Elapsed();

  // Print the results.
  m_par.Print();
  m_par.WriteImage();

  // Run the serial version.
  dpc_common::TimeInterval t_ser;
  m_ser.Evaluate();
  double serial_time = t_ser.Elapsed();

  // Report the results.
  cout << std::setw(20) << "Serial time: " << serial_time << "s\n";
  cout << std::setw(20) << "Parallel time: " << (parallel_time / repetitions)
       << "s\n";

  // Validate.
  m_par.Verify(m_ser);
}

int main(int argc, char *argv[]) {
  try {
    // Create a queue on the default device. Set SYCL_DEVICE_TYPE environment
    // variable to (CPU|GPU|FPGA|HOST) to change the device.
    //queue q(default_selector{}, dpc_common::exception_handler);
      queue q(gpu_selector{}, dpc_common::exception_handler);

    // Display the device info.
    ShowDevice(q);

    // Compute Mandelbrot set.
    Execute(q);
  } catch (...) {
    // Some other exception detected.
    cout << "Failed to compute Mandelbrot set.\n";
    std::terminate();
  }

  cout << "Successfully computed Mandelbrot set.\n";
  return 0;
}


Overwriting lab/simple.cpp


### Build and Run
Select the cell below and click Run ▶ to compile and execute the code above:

In [1]:
! chmod 755 q; chmod 755 run_simple.sh;if [ -x "$(command -v qsub)" ]; then ./q run_simple.sh; else ./run_simple.sh; fi

Job has been submitted to Intel(R) DevCloud and will execute soon.

 If you do not see result in 60 seconds, please restart the Jupyter kernel:
 Kernel -> 'Restart Kernel and Clear All Outputs...' and then try again

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
798175.v-qsvr-1            ...ub-singleuser u60339          00:00:20 R jupyterhub     
798187.v-qsvr-1            run_simple.sh    u60339                 0 Q batch          

Waiting for Output ██████████████████████ Done⬇

########################################################################
#      Date:           Sat 20 Feb 2021 04:28:19 AM PST
#    Job ID:           798187.v-qsvr-1.aidevcloud
#      User:           u60339
# Resources:           neednodes=1:gpu:ppn=2,nodes=1:gpu:ppn=2,walltime=06:00:00
########################################################################

## u60339 is compiling DPCPP_Essentials Modu

_If the Jupyter cells are not responsive or if they error out when you compile the code samples, please restart the Jupyter Kernel: 
"Kernel->Restart Kernel and Clear All Outputs" and compile the code samples again_

## How to Compile & Run DPC++ program

Overwriting lab/simple-vector-incr.cpp


Job has been submitted to Intel(R) DevCloud and will execute soon.

 If you do not see result in 60 seconds, please restart the Jupyter kernel:
 Kernel -> 'Restart Kernel and Clear All Outputs...' and then try again

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
798175.v-qsvr-1            ...ub-singleuser u60339          00:01:02 R jupyterhub     
798230.v-qsvr-1            ...ector-incr.sh u60339                 0 Q batch          

Waiting for Output █████████████████████████████████████

_If the Jupyter cells are not responsive or if they error out when you compile the code samples, please restart the Jupyter Kernel: 
"Kernel->Restart Kernel and Clear All Outputs" and compile the code samples again_

# Summary
In this module you will have learned the following:
* How oneAPI solves the challenges of programming in a heterogeneous world 
* Take advantage of oneAPI solutions to enable your workflows

